In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import requests
import json
import pandas as pd
import time

In [65]:
game_df = pd.read_csv('/content/drive/MyDrive/DS5230_Project/top_games_100.csv')

In [66]:
game_df.head()

,id,name,box_art_url
0,509658,Just Chatting,https://static-cdn.jtvnw.net/ttv-boxart/509658...
1,21779,League of Legends,https://static-cdn.jtvnw.net/ttv-boxart/21779-...
2,32982,Grand Theft Auto V,https://static-cdn.jtvnw.net/ttv-boxart/32982_...
3,33214,Fortnite,https://static-cdn.jtvnw.net/ttv-boxart/33214-...
4,518203,Sports,https://static-cdn.jtvnw.net/ttv-boxart/518203...


In [71]:
import requests

def get_access_token(client_id, client_secret):
    """ Get the access token using client credentials. """
    body = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post('https://id.twitch.tv/oauth2/token', data=body)
    return response.json()['access_token']

def get_game_genre(game_name, client_id, access_token):
    """ Retrieve the genre of a game using the IGDB API. """
    headers = {
        'Client-ID': client_id,
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    body = f'''
    search "{game_name}";
    fields name, genres.name;
    where name="{game_name}";
    limit 1;
    '''
    response = requests.post('https://api.igdb.com/v4/games', headers=headers, data=body)
    if response.status_code == 200 and response.json():
        game_data = response.json()[0]
        if 'genres' in game_data:
            return [genre['name'] for genre in game_data['genres']]
        else:
            return 'No genre found'
    return 'Game not found'


In [69]:
client_id = "CLIENT_ID"
client_secret = "CLIENT_SECRET"
access_token = get_access_token(client_id, client_secret)

In [70]:
game_name = 'League of Legends'
genre = get_game_genre(game_name, client_id, access_token)
print(f"The genre of {game_name} is {clean_name(genre)}")

The genre of League of Legends is roleplaying rpg strategy moba


In [113]:
games_genre = dict()
for game in game_df['name']:
  games_genre[game] = get_game_genre(game, client_id, access_token)

In [114]:
game_df['name']

0                     Just Chatting
1                 League of Legends
2                Grand Theft Auto V
3                          Fortnite
4                            Sports
                  ...              
95                           Metin2
96                Honkai: Star Rail
97                I'm Only Sleeping
98    Software and Game Development
99             Magic: The Gathering
Name: name, Length: 100, dtype: object

In [115]:
for key, value in games_genre.items():
  if value == 'No genre found' or value == "Game not found":
    print(key)

Just Chatting
Grand Theft Auto V
VALORANT
Music
ELDEN RING
Special Events
Slots
PUBG: BATTLEGROUNDS
Travel & Outdoors
UNDERTALE
Chess
Art
ASMR
HELLDIVERS 2
FINAL FANTASY XIV ONLINE
Age of Empires II
ARK: Survival Ascended
StarCraft II
EVE Online
Always On
Reksio i Skarb Piratów
ROBLOX
Games + Demos
TEKKEN 8
Animals, Aquariums, and Zoos
Yu-Gi-Oh! TRADING CARD GAME
Pools, Hot Tubs, and Beaches
SMITE
SCUM
Software and Game Development


In [116]:
games_genre["Just Chatting"] = ["Indie", "Simulator", "Adventure"]
games_genre["Grand Theft Auto V"] = ['Shooter', 'Role-playing (RPG)', 'Adventure', "Hack and slash/Beat 'em up"]
games_genre["VALORANT"] = ["Strategy", "Shooter", "MOBA"]
games_genre["ELDEN RING"] = ["Role-playing (RPG)", "Adventure", "Fantasy"]
games_genre["PUBG: BATTLEGROUNDS"] = ["Shooter", "Strategy", "MOBA"]
games_genre["Chess"] = ["Strategy", "Card & Board Game"]
games_genre["UNDERTALE"] = ["Indie", "Role-playing (RPG)", "Adventure"]
games_genre["HELLDIVERS 2"] = ["Indie", "Shooter"]
games_genre["FINAL FANTASY XIV ONLINE"] = ["Role-playing (RPG)"]
games_genre["Age of Empires II"] = ["Strategy"]
games_genre["ARK: Survival Ascended"] = ["Shooter", "Adventure", 'Role-playing (RPG)', "indie"]
games_genre["StarCraft II"] = ["Adventure", "Strategy"]
games_genre["EVE Online"] = ["Role-playing (RPG)", "Strategy", "Simulator"]
games_genre["Reksio i Skarb Piratów"] = ["Shooter", "Simulator", "Role-playing (RPG)"]
games_genre["ROBLOX"] = ["Shooter", "Simulator", "Role-playing (RPG)"]
games_genre["TEKKEN 8"] = ["Adventure"]
games_genre["Yu-Gi-Oh! TRADING CARD GAME"] = ["Card & Board Game", "Strategy"]
games_genre["SMITE"] = ["Adventure", 'Role-playing (RPG)', "Strategy", "MOBA"]
games_genre["SCUM"] = ["Adventure", "Indie"]


In [117]:
for key, value in games_genre.items():
  if value == 'No genre found' or value == "Game not found":
    games_genre[key] = ["NA"]

In [102]:
max_length = max(len(items) for items in games_genre.values())

In [104]:
# Pad the shorter lists with `None`
for key, values in games_genre.items():
    if len(values) < max_length:
        games_genre[key].extend([None] * (max_length - len(values)))

In [105]:
games_genre_df = df = pd.DataFrame(games_genre)

In [119]:
games_genre_df = pd.DataFrame(columns=['game_name', 'genre'])
rows = []
for game_name, genre in games_genre.items():
    # Each row is a dictionary that matches the DataFrame structure
    rows.append({'game_name': game_name, 'genre': genre})

# Concatenate all rows at once to the DataFrame
games_genre_df = pd.concat([games_genre_df, pd.DataFrame(rows)], ignore_index=True)


                        game_name  \
0                   Just Chatting   
1               League of Legends   
2              Grand Theft Auto V   
3                        Fortnite   
4                          Sports   
..                            ...   
95                         Metin2   
96              Honkai: Star Rail   
97              I'm Only Sleeping   
98  Software and Game Development   
99           Magic: The Gathering   

                                                genre  
0                       [Indie, Simulator, Adventure]  
1                [Role-playing (RPG), Strategy, MOBA]  
2   [Shooter, Role-playing (RPG), Adventure, Hack ...  
3                                           [Shooter]  
4                                             [Sport]  
..                                                ...  
95                    [Role-playing (RPG), Adventure]  
96  [Role-playing (RPG), Turn-based strategy (TBS)...  
97                       [Point-and-click, Adventur

In [123]:
games_genre_df.head()

,game_name,genre
0,Just Chatting,"[Indie, Simulator, Adventure]"
1,League of Legends,"[Role-playing (RPG), Strategy, MOBA]"
2,Grand Theft Auto V,"[Shooter, Role-playing (RPG), Adventure, Hack ..."
3,Fortnite,[Shooter]
4,Sports,[Sport]


In [122]:
games_genre_df.to_csv('/content/drive/My Drive/DS5230_Project/top_100_games_genre.csv', index=False)